In [1]:
import geoengine as ge
from geoengine.workflow_builder.operators import GdalSource, TemporalRasterAggregation, RasterStacker, RenameBands, \
    Expression,RasterTypeConversion, RasterBandDescriptor
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
ge.initialize("http://localhost:3030/api")

In [3]:
marburg_center_utm = [483843.45952547, 5628614.821735]
bbox = ge.BoundingBox2D(
    xmin=marburg_center_utm[0] - 10 * 512,
    xmax=marburg_center_utm[0] + 10 * 512,
    ymin=marburg_center_utm[1] - 10 * 512,
    ymax=marburg_center_utm[1] + 10 * 512,
)
query_rectangle=ge.QueryRectangle(
    spatial_bounds=bbox,
    time_interval=ge.TimeInterval(np.datetime64("2022-07-01T00:00:00")),
    srs='EPSG:32632',
    resolution=ge.SpatialResolution(10, 10),
)

In [4]:
workflow = TemporalRasterAggregation(
    aggregation_type='max',
    granularity='days',
    window_size=3,
    ignore_no_data=True,
    source=Expression(
        expression='''
            if C >= 8 {
                NODATA
            } else if (A-B)/(A+B) > 0.3 {
                1
            } else {
                0
            }
        ''',
        output_type='U8',
        output_band=RasterBandDescriptor(
            "waterbodies",
            measurement=ge.ClassificationMeasurement(
                "water bodies",
                classes={
                    0: 'no water',
                    1: 'water',
                }
            ),
        ),
        source=
            RasterStacker(
                sources=[
                    GdalSource("_:5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5:`UTM32N:B03`"),
                    GdalSource("_:5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5:`UTM32N:B08`"),
                    RasterTypeConversion(
                        GdalSource("_:5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5:`UTM32N:SCL`"),
                        output_data_type='U16',
                    ),
                ],
            ),
    ),
)

print(json.dumps(workflow.to_workflow_dict(), indent=2))

workflow = ge.register_workflow(workflow)

workflow

{
  "type": "Raster",
  "operator": {
    "type": "TemporalRasterAggregation",
    "params": {
      "aggregation": {
        "type": "max",
        "ignoreNoData": true,
        "percentile": null
      },
      "window": {
        "granularity": "days",
        "step": 3
      },
      "windowReference": null,
      "outputType": null
    },
    "sources": {
      "raster": {
        "type": "Expression",
        "params": {
          "expression": "\n            if C >= 8 {\n                NODATA\n            } else if (A-B)/(A+B) > 0.3 {\n                1\n            } else {\n                0\n            }\n        ",
          "outputType": "U8",
          "mapNoData": false,
          "outputBand": {
            "measurement": {
              "classes": {
                "0": "no water",
                "1": "water"
              },
              "measurement": "water bodies",
              "type": "classification"
            },
            "name": "waterbodies"
          

a4391791-82b5-5451-a5a9-f771ab3ad5e0

In [5]:

data_array = workflow.wms_get_map_as_image(
    bbox=query_rectangle,
    colorizer=ge.Colorizer.linear_with_mpl_cmap('viridis', (0, 3))
)

data_array

b'\n<?xml version="1.0" encoding="UTF-8"?>\n    <ServiceExceptionReport version="1.3.0" xmlns="http://www.opengis.net/ogc" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/ogc https://cdc.dwd.de/geoserver/schemas/wms/1.3.0/exceptions_1_3_0.xsd">\n    <ServiceException>\n        Serde json error: unknown variant `linearGradient`, expected `singleBand` at line 1 column 642\n    </ServiceException>\n</ServiceExceptionReport>'


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7b57ed0c62f0>